In [ ]:
with open("input.txt") as f:
    lines = f.readlines()
lines = [l.strip() for l in lines]
lines[:5]

In [ ]:
# split into input patterns
patterns = []
pattern = []
for line in lines:
    if line:
        pattern.append(line)
    else:
        patterns.append(pattern)
        pattern = []
if pattern:
    patterns.append(pattern)
patterns

In [ ]:
def get_col_tuples(pattern):
    tuple_list = []
    for i in range(0, len(pattern[0])):
        col_list = []
        for j in range(0, len(pattern)):
            col_list.append(pattern[j][i])
        tuple_list.append(tuple(col_list))
    return tuple_list

In [ ]:
pattern = patterns[1]
row_tuples = [tuple(r) for r in pattern]
col_tuples = get_col_tuples(pattern)
# col_tuples

In [ ]:
# I misunderstood the problem first and didn't realize, reflections need to go to the boundary and was looking for the longest reflection
# Instead of rewriting, the code was adapted until it fit, so it may be a bit weird

In [ ]:
def find_refl_length(i, tuple_list):
    refl_len = 0
    while True:
        if (i - (refl_len) < 0) or (i + refl_len + 1 >= len(tuple_list)):
            break
        # print(f"{i-(refl_len)},{i+refl_len+1}")
        if tuple_list[i - (refl_len)] == tuple_list[i + refl_len + 1]:
            refl_len += 1
        else:
            break
    return refl_len

In [ ]:
# Return a 4-tuple with following values
# - 0 horizontal 1 vertical reflection line
# - length of reflection
# - start index of reflection
# - length of field in perpendicular direction of reflection line (thought I need this, turns out I don't but I kept it in)
def find_refl(pattern):
    row_tuples = [tuple(r) for r in pattern]
    col_tuples = get_col_tuples(pattern)

    hor_refl = (0, 0, 0, 0)
    for i in range(0, len(row_tuples) - 1):
        if row_tuples[i] == row_tuples[i + 1]:
            refl_len = find_refl_length(i, row_tuples)
            # New reflection is longer (in case there might be 2, I don't think there ever are)
            # And it ends at either lower or upper end of the field 
            if refl_len > hor_refl[1] and (
                (i - (refl_len - 1) == 0) or (i + refl_len == len(row_tuples) - 1)
            ):
                hor_refl = (0, refl_len, i, len(row_tuples))

    vert_refl = (1, 0, 0, 0)
    for i in range(0, len(col_tuples) - 1):
        if col_tuples[i] == col_tuples[i + 1]:
            refl_len = find_refl_length(i, col_tuples)
            if refl_len > vert_refl[1] and (
                (i - (refl_len - 1) == 0) or (i + refl_len == len(col_tuples) - 1)
            ):
                vert_refl = (1, refl_len, i, len(col_tuples))

    if vert_refl[1] > hor_refl[1]:
        return vert_refl
    else:
        return hor_refl


# With index of reflection start, lines above or left to it are always 0->index
def get_refl_value(refl):
    if refl[0] == 1:
        return refl[2] + 1
    else:
        return 100 * (refl[2] + 1)

In [ ]:
sum([get_refl_value(find_refl(p)) for p in patterns])

part 2

In [ ]:
# Lets try brute forcing this by just smudging every single possible field

In [ ]:
def apply_smudge_row(i, row):
    if row[i] == "#":
        return row[:i] + "." + row[i + 1 :]
    else:
        return row[:i] + "#" + row[i + 1 :]


def apply_smudge(i, j, pattern: list[str]):
    pattern = pattern.copy()
    pattern[i] = apply_smudge_row(j, pattern[i])
    return pattern

In [ ]:
# Add filter for old reflection
def find_refl_smudge(pattern, old_refl):
    row_tuples = [tuple(r) for r in pattern]
    col_tuples = get_col_tuples(pattern)

    hor_refl = (0, 0, 0, 0)
    for i in range(0, len(row_tuples) - 1):
        if row_tuples[i] == row_tuples[i + 1]:
            refl_len = find_refl_length(i, row_tuples)
            if refl_len > hor_refl[1] and (
                (i - (refl_len - 1) == 0) or (i + refl_len == len(row_tuples) - 1)
            ):
                if (0, refl_len, i, len(row_tuples)) != old_refl:
                    hor_refl = (0, refl_len, i, len(row_tuples))

    vert_refl = (1, 0, 0, 0)
    for i in range(0, len(col_tuples) - 1):
        if col_tuples[i] == col_tuples[i + 1]:
            refl_len = find_refl_length(i, col_tuples)
            if refl_len > vert_refl[1] and (
                (i - (refl_len - 1) == 0) or (i + refl_len == len(col_tuples) - 1)
            ):
                if (1, refl_len, i, len(col_tuples)) != old_refl:
                    vert_refl = (1, refl_len, i, len(col_tuples))

    if vert_refl[1] > hor_refl[1]:
        return vert_refl
    else:
        return hor_refl

In [ ]:
find_refl(apply_smudge(0, 0, patterns[0]))

In [ ]:
def smudge_search(pattern):
    old_refl = find_refl(pattern)
    # print(pattern)
    for i in range(0, len(pattern)):
        for j in range(0, len(pattern[0])):
            p = apply_smudge(i, j, pattern)
            refl = find_refl_smudge(p, old_refl)
            if refl != (0, 0, 0, 0):
                # print((i,j))
                # print(refl)
                # print(p)
                return refl


smudge_search(patterns[0])

In [ ]:
sum([get_refl_value(smudge_search(p)) for p in patterns])